# Course1-Project1: EdTech Backend System

In [49]:
import hashlib
#from enum import Enum

#class Role(Enum):
#    LEARNER = "Learner"
#    INSTRUCTOR = "Instructor"
        
class User:
    counter = 0
    
    def __init__(self, name, role, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'learner' or 'instructor'
        'email' and 'password' are optional.
        '''
        #if not isinstance(role,Role):
        #    raise ValueError("role must be a member of Role Enum.")
            
        self.name = name
        self.set_id(name)
        self.role = role
        self.email = email
        #self.__password_hash = None OJO: No tengo que declarar ni definir todos los atr. en el contructor. Se definen por primera vez en los mét. que los usan

        if password:
            self.hash_password(password)
            self.dotify_password(password)

        User.counter +=1

    def set_id(self,name):
        # Encode the string and compute a hash (SHA-256)
        hash_object = hashlib.sha256(name.encode('utf-8'))
        # Convert the hexadecimal hash to an integer
        self.id = int(hash_object.hexdigest(), 16)
        # Take only part of it (e.g., last 10 digits)
        self.id = self.id % (10**10)
        
    def set_email(self, email):
        self.email=email

    def hash_password(self, password):
        '''
        Hash the password using SHA-256 before storing it
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed
        
    def dotify_password(self, password):
        '''
        Mask the password with dots
        '''
        self.dotified_password = '.' * len(password)

    def validate_password(self,password):
        """
        Check if a given password matches the stored hashed password.
        """
        if self.__password_hash is None:
            return False
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        return hashed == self.__password_hash
        
    def validate_email(self,email):
        return self.email == email            

    def set_role(self):
        option = int(input("Choose either (1) for 'Learner' or (2) for 'Instructor': "))
        if option == 1:
            self.role = "Learner"
        elif option == 2:
            self.role = "Instructor"
        else:
            raise ValueError("Wrong value")
            
    def print_info(self):
        print("Name    :", self.name)
        print("Id      :", self.id)
        print("Role    :", self.role.value)
        print("Email   :", self.email)
        print("Password:", self.dotified_password)

class SlttechBackend(User):
    # no requiere constructor, pero tengo que 'override' porque hereda de User que sí requiere parámetros de entrada
    def __init__(self):
        pass
    
    #Version 1: Simple add_user method. Positive case where all parameters are always given. No checks. Password as readable text
    # Version 2: Only name and role are mandatory attr
    # Version 3: Type in password, but mask it
    # Version 4: Check for proper email syntax - call validate-email?
    def add_user(self): #v1
        print("Add User")
        name = input('Full name:')
        print("Select role for", name)
        role = self.set_role() # es un mét heredado de User
        email = input('Email:')
        password = input('Password:')

        usr = User(name, role, email, password)
    
    def list_users(self):
        pass
        
    def manage_users(self):
        print("""STL Tech Backend
        Manage Users Method.
        Options:
            (1) - Add users
            (2) - List users
            (0) - Exit        
        """)
        while True:
            option = int(input('Select option: '))
            match option:
                case 1:
                    self.add_user()
                case 2:
                    print("List users")
                    self.list_users()
                case 0:
                    print("Exit method. Bye!")
                    break

            


# declaración programática de usuarios. Testing purpose
#u1 = User("Alice", Role.LEARNER,"alice@gmail.com","1234")
#u1.print_info()
#u2 = User("Bob",Role.INSTRUCTOR)
#u1.print_info()
#u2.set_email("bob@gmail.com")
#u2.print_info()
#if (not u2.validate_email("bob2@example.com")):
#    print("Wrong email")

#u2.set_password("1234")
#if not u2.validate_password("1234"):
#    print("Wrong password")
#else:
#    print("Password match")
#help(u1)

be = SlttechBackend()
be.manage_users()


STL Tech Backend
        Manage Users Method.
        Options:
            (1) - Add users
            (2) - List users
            (0) - Exit        
        


Select option:  1


Add User


Full name: c


Select role for c


Choose either (1) for 'Learner' or (2) for 'Instructor':  1
Email: e
Password: e
Select option:  0


Exit method. Bye!
